In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from pprint import pprint
FILE_FOLDER = '../prediction_data/%s'
AFTER_FILE_FOLDER = '../prediction_data/afterAnalysis/%s'

# Mean Filling

In [10]:
aq_17_18 = pd.read_csv(FILE_FOLDER % 'airQuality_201701-201801.csv')
print(aq_17_18.columns)
print('total records:',len(aq_17_18))
print('NAN of O3:',len(aq_17_18.loc[aq_17_18['PM10'].isnull()]))


aq18_02_03 = pd.read_csv(FILE_FOLDER % 'airQuality_201802-201803.csv')
print(aq18_02_03.columns)
print('total records:',len(aq18_02_03))
print('NAN of O3:',len(aq18_02_03.loc[aq18_02_03['PM10'].isnull()]))


aq18_04 = pd.read_csv(FILE_FOLDER % 'aiqQuality_201804.csv')
print(aq18_04.columns)
print('total records:',len(aq18_04))
print('NAN of O3:',len(aq18_04.loc[aq18_04['PM10_Concentration'].isnull()]))

Index(['stationId', 'utc_time', 'PM2.5', 'PM10', 'NO2', 'CO', 'O3', 'SO2'], dtype='object')
total records: 311010
NAN of O3: 83263
Index(['stationId', 'utc_time', 'PM2.5', 'PM10', 'NO2', 'CO', 'O3', 'SO2'], dtype='object')
total records: 49420
NAN of O3: 12912
Index(['id', 'station_id', 'time', 'PM25_Concentration', 'PM10_Concentration',
       'NO2_Concentration', 'CO_Concentration', 'O3_Concentration',
       'SO2_Concentration'],
      dtype='object')
total records: 23310
NAN of O3: 4885


In [11]:
need_features = ['station_id', 'time', 'PM25', 'PM10', 'O3']
aq_17_18 = aq_17_18.rename(index=str, columns={"stationId": "station_id","utc_time": "time", "PM2.5": "PM25"})
aq18_02_03 = aq18_02_03.rename(index=str, columns={"stationId": "station_id","utc_time": "time", "PM2.5": "PM25"})
aq18_04 = aq18_04.rename(index=str, columns={"PM25_Concentration": "PM25",
                                             "PM10_Concentration":"PM10",
                                             "O3_Concentration":"O3"})
aq_17_18 = aq_17_18[need_features]
aq18_02_03 = aq18_02_03[need_features]
aq18_04 = aq18_04[need_features]
aq18_04.head()

,station_id,time,PM25,PM10,O3
0,dongsi_aq,2018-04-01 02:00:00,259.0,NaN,56.0
1,tiantan_aq,2018-04-01 02:00:00,250.0,NaN,64.0
2,guanyuan_aq,2018-04-01 02:00:00,240.0,246.0,49.0
3,wanshouxigong_aq,2018-04-01 02:00:00,255.0,260.0,65.0
4,aotizhongxin_aq,2018-04-01 02:00:00,266.0,NaN,45.0


In [18]:
df_aq = aq_17_18.append(aq18_02_03)
# df_aq = aq18_02_03
df_aq = df_aq.append(aq18_04)
df_aq = df_aq.reset_index()
df_aq['datetime'] = pd.to_datetime(df_aq['time'])
df_aq_0104 = df_aq[df_aq['datetime'] >= '2018-01-01 00:00:00']
df_aq_0104 = df_aq_0104[need_features]

len(df_aq_0104)

98490

In [15]:
aq_station_name = ['dongsi_aq', 'tiantan_aq', 'guanyuan_aq', 'wanshouxigong_aq',
                   'aotizhongxin_aq', 'nongzhanguan_aq', 'wanliu_aq', 'beibuxinqu_aq', 
                   'zhiwuyuan_aq', 'fengtaihuayuan_aq', 'yungang_aq', 'gucheng_aq', 
                   'fangshan_aq', 'daxing_aq', 'yizhuang_aq', 'tongzhou_aq',
                   'shunyi_aq', 'pingchang_aq', 'mentougou_aq', 'pinggu_aq', 
                   'huairou_aq', 'miyun_aq', 'yanqin_aq', 'dingling_aq', 'badaling_aq', 
                   'miyunshuiku_aq', 'donggaocun_aq', 'yongledian_aq', 'yufa_aq', 
                   'liulihe_aq', 'qianmen_aq', 'yongdingmennei_aq', 'xizhimenbei_aq',
                   'nansanhuan_aq', 'dongsihuan_aq']
len(aq_station_name)

35

## Fill the Time Series (Time) Information

In [11]:
def arrange_aq_data(original_df,start_time,end_time):
    pp = ["PM25","PM10","O3"]
    df_aq_copy = original_df.copy()
    df_aq_copy.set_index(['station_id','time'], inplace=True)
    
    making_dict = {
        'station_id':[],
        'time':[],
        'PM25':[],
        "PM10":[],
        "O3":[]
    }
    time_span = list(pd.date_range(start = start_time, end = end_time, closed=None, freq='H'))
    time_span = list(map(lambda x:str(x),time_span))
    for c_time in tqdm(time_span,total = len(time_span)):
        for c_station in aq_station_name:
            making_dict['station_id'].append(c_station)
            making_dict['time'].append(c_time)
            try:
                ele = df_aq_copy.loc[(c_station,c_time)]
                for p in pp:
                    making_dict[p].append(ele[p].mean())
            except:
                for p in pp:
                    making_dict[p].append(np.nan)
    new_df = pd.DataFrame.from_dict(making_dict)
    return new_df
    
# newwww_df =  arrange_aq_data(df_aq_0104,'2018-01-01 00:00:00','2018-04-30 23:00:00')


In [16]:
df_aq_17 = pd.read_csv(FILE_FOLDER % 'airQuality_201701-201801.csv')
df_aq_18_0203 = pd.read_csv(FILE_FOLDER % 'airQuality_201802-201803.csv')
df_aq_17_1803 = df_aq_17.append(df_aq_18_0203)
df_aq_17_1803 = df_aq_17_1803[['stationId', 'utc_time', 'PM2.5', 'PM10','O3']]

df_aq_17_1803.rename(index=str, columns={'stationId':'station_id','utc_time':'time','PM2.5':'PM25'},inplace = True)

arranged_aq_17_1803 =  arrange_aq_data(df_aq_17_1803,'2017-01-01 00:00:00','2018-03-31 23:00:00')
# len(list(pd.date_range(start = '2018-01-01 00:00:00', end = '2018-04-30 23:00:00', closed=None, freq='H')))

100%|██████████| 10920/10920 [13:06<00:00, 13.89it/s]


# Filling One Station by others' mean in the same time

In [18]:
def fill_missing_data(original_df):
    pp = ["PM25","PM10","O3"]
    df_aq_copy = original_df.copy()
    df_aq_copy.set_index(['station_id','time'], inplace=True)
    # pp = ["PM25"]
    for p in pp:
        df_NOT_NAN = original_df.loc[original_df[p].isnull() == False]
        df_NAN = original_df.loc[original_df[p].isnull()]
    #     print(len(df_NAN),len(df_NOT_NAN))
        df_NAN.set_index(['station_id','time'], inplace=True)
        miss_aq_name = set()
        for i in df_NAN.index.values.tolist():
            miss_aq_name.add(i[0])
        miss_aq_name = list(miss_aq_name)

        for aq_name in tqdm(miss_aq_name,total = len(miss_aq_name)):
            for time in df_NAN.loc[aq_name].index.values.tolist():
                mean_value = df_NOT_NAN[df_NOT_NAN['time'] == time][p].mean()
                df_aq_copy.at[(aq_name,time),p] = mean_value
                
    return df_aq_copy.reset_index()

In [19]:
new_aq_17_1803 = fill_missing_data(arranged_aq_17_1803)

100%|██████████| 35/35 [20:43<00:00, 32.73s/it]


In [21]:
pp = ["PM25","PM10","O3"]
for p in pp:
    m_n = len(arranged_aq_17_1803.loc[arranged_aq_17_1803[p].isnull()])
    print('Missing num of %s: '%p, m_n)
print('after filling')
pp = ["PM25","PM10","O3"]
for p in pp:
    m_n = len(new_aq_17_1803.loc[new_aq_17_1803[p].isnull()])
    print('Missing num of %s: '%p, m_n)

Missing num of PM25:  51079
Missing num of PM10:  123025
Missing num of O3:  51390
after filling
Missing num of PM25:  28630
Missing num of PM10:  31675
Missing num of O3:  28630


In [26]:
arranged_aq_17_1803[880:900]

,station_id,time,PM25,PM10,O3
880,nongzhanguan_aq,2017-01-02 01:00:00,13.0,NaN,19.0
881,wanliu_aq,2017-01-02 01:00:00,28.0,NaN,11.0
882,beibuxinqu_aq,2017-01-02 01:00:00,140.0,NaN,5.0
883,zhiwuyuan_aq,2017-01-02 01:00:00,17.0,NaN,53.0
884,fengtaihuayuan_aq,2017-01-02 01:00:00,373.0,399.0,9.0
885,yungang_aq,2017-01-02 01:00:00,327.0,NaN,6.0
886,gucheng_aq,2017-01-02 01:00:00,28.0,NaN,14.0
887,fangshan_aq,2017-01-02 01:00:00,289.0,NaN,6.0
888,daxing_aq,2017-01-02 01:00:00,314.0,343.0,7.0
889,yizhuang_aq,2017-01-02 01:00:00,246.0,261.0,8.0


In [15]:
def fill_missing_for_another(df_after,df_target):
    pp = ["PM25","PM10","O3"]
    df_aq_copy = df_target.copy()
    df_aq_copy.set_index(['station_id','time'], inplace=True)
    df_after_copy = df_after.set_index(['station_id','time'], inplace=True)
    # pp = ["PM25"]
    for p in pp:
        df_NAN = df_target.loc[df_target[p].isnull()]
    #     print(len(df_NAN),len(df_NOT_NAN))
        df_NAN.set_index(['station_id','time'], inplace=True)
        miss_aq_name = set()
        for i in df_NAN.index.values.tolist():
            miss_aq_name.add(i[0])
        miss_aq_name = list(miss_aq_name)

        for aq_name in tqdm(miss_aq_name,total = len(miss_aq_name)):
            for time in df_NAN.loc[aq_name].index.values.tolist():
                mean_value = df_after_copy.loc[aq_name].loc[time][p]
                df_aq_copy.set_value((aq_name,time),p,mean_value)
    return df_aq_copy.reset_index()

In [39]:
df_after_04 = pd.read_csv(AFTER_FILE_FOLDER % 'aq18_02_04.csv')
df_after_ = pd.read_csv(AFTER_FILE_FOLDER % 'aq17_03_07.csv')
df_after_ = new_d.append(df_after_04,sort=False)

df_train = pd.read_csv(AFTER_FILE_FOLDER % 'train_data.csv')

In [40]:
pp = ["PM25","PM10","O3"]
for p in pp:
    m_n = len(df_train.loc[df_train[p].isnull()])
    print('Missing num of %s: '%p, m_n)

Missing num of PM25:  10483
Missing num of PM10:  38709
Missing num of O3:  9713


In [41]:
def fill_missing_for_another(df_after,df_target):
    pp = ["PM25","PM10","O3"]
    df_aq_copy = df_target.copy()
    df_aq_copy.set_index(['station_id','time'], inplace=True)
    df_after_copy = df_after.set_index(['station_id','time'])
    # pp = ["PM25"]
    for p in pp:
        df_NAN = df_target.loc[df_target[p].isnull()]
    #     print(len(df_NAN),len(df_NOT_NAN))
        df_NAN.set_index(['station_id','time'], inplace=True)
        miss_aq_name = set()
        for i in df_NAN.index.values.tolist():
            miss_aq_name.add(i[0])
        miss_aq_name = list(miss_aq_name)

        for aq_name in tqdm(miss_aq_name,total = len(miss_aq_name)):
            for time in df_NAN.loc[aq_name].index.values.tolist():
                mean_value = df_after_copy.loc[aq_name].loc[time][p].mean()
#                 pprint(mean_value)
                df_aq_copy.set_value((aq_name,time),p,mean_value)

    return df_aq_copy.reset_index()

In [42]:
# new_train_d = fill_missing_for_another(df_after_,df_train)
df_train = df_train.rename(index=str, columns={"aq_station_id": "station_id"})

In [43]:
new_train_d = fill_missing_for_another(df_after_,df_train)


  0%|          | 0/35 [00:00<?, ?it/s]/Users/haomao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

100%|██████████| 35/35 [00:18<00:00,  2.47it/s]

100%|██████████| 35/35 [01:05<00:00,  1.65s/it]

100%|██████████| 35/35 [00:16<00:00,  1.71it/s]


In [44]:
pp = ["PM25","PM10","O3"]
for p in pp:
    m_n = len(new_train_d.loc[new_train_d[p].isnull()])
    print('Missing num of %s: '%p, m_n)

Missing num of PM25:  35
Missing num of PM10:  2310
Missing num of O3:  35


In [47]:
new_train_d.to_csv(AFTER_FILE_FOLDER % 'train_data_after_first_filling.csv')

In [46]:
len(df_train)

153160

# Machine Learning Filling

## PM25

In [ ]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))


## PM10

## O3